In [1]:
from snowflake.snowpark import Session
# create_temp_table warning suppresion
import warnings; warnings.simplefilter('ignore')
import configparser

import pandas as pd

In [32]:
config = configparser.ConfigParser()
config.read("snowflake_connection.ini")

['snowflake_connection.ini']

In [33]:
connection_parameters = {
    "user": f'{config["Snowflake"]["user"]}',
    "password": f'{config["Snowflake"]["password"]}',
    #"password": os.getenv('snowflake_password'),
    "account": f'{config["Snowflake"]["account"]}',
    #"account": os.getenv('snowflake_account'),
    "WAREHOUSE": f'{config["Snowflake"]["WAREHOUSE"]}',
    "DATABASE": f'{config["Snowflake"]["DATABASE"]}',
    "SCHEMA": f'{config["Snowflake"]["SCHEMA"]}'
}

In [34]:
def snowflake_connector(conn):
    try:
        session = Session.builder.configs(conn).create()
        print("connection successful!")
    except:
        raise ValueError("error while connecting with db")
    return session

session = snowflake_connector(connection_parameters)

connection successful!


In [48]:
df =  pd.read_csv('/data/Lumin_Dump_CreditRiskAnalytics_v1.csv')

In [49]:
df['APPL_MONTH'] = pd.to_datetime(df['APPL_MONTH'], format='ISO8601')

In [50]:
df['HOUSETYPE_MODE'].fillna('Missing', inplace=True)
df['WALLSMATERIAL_MODE'].fillna('Missing', inplace=True)
df['EMERGENCYSTATE_MODE'].fillna('Missing', inplace=True)
df['OCCUPATION_TYPE'].fillna('Missing', inplace=True)
df['FONDKAPREMONT_MODE'].fillna('Missing', inplace=True)
df['NAME_TYPE_SUITE'].fillna('Missing', inplace=True)

In [51]:
df['RECORD_COUNT'] = df['RECORD_COUNT'].apply(str)
df['APPL_MONTH'] = df['APPL_MONTH'].apply(str)
df['SK_ID_CURR'] = df['SK_ID_CURR'].apply(str)
df['TARGET'] = df['TARGET'].apply(str)
df['FLAG_MOBIL'] = df['FLAG_MOBIL'].apply(str)
df['FLAG_EMP_PHONE'] = df['FLAG_EMP_PHONE'].apply(str)
df['FLAG_WORK_PHONE'] = df['FLAG_WORK_PHONE'].apply(str)
df['FLAG_CONT_MOBILE'] = df['FLAG_CONT_MOBILE'].apply(str)
df['FLAG_PHONE'] = df['FLAG_PHONE'].apply(str)
df['FLAG_EMAIL'] = df['FLAG_EMAIL'].apply(str)
df['REGION_RATING_CLIENT_W_CITY'] = df['REGION_RATING_CLIENT_W_CITY'].apply(str)
df['HOUR_APPR_PROCESS_START'] = df['HOUR_APPR_PROCESS_START'].apply(str)
df['REG_REGION_NOT_LIVE_REGION'] = df['REG_REGION_NOT_LIVE_REGION'].apply(str)
df['REG_REGION_NOT_WORK_REGION'] = df['REG_REGION_NOT_WORK_REGION'].apply(str)
df['LIVE_REGION_NOT_WORK_REGION'] = df['LIVE_REGION_NOT_WORK_REGION'].apply(str)
df['REG_CITY_NOT_LIVE_CITY'] = df['REG_CITY_NOT_LIVE_CITY'].apply(str)
df['REG_CITY_NOT_WORK_CITY'] = df['REG_CITY_NOT_WORK_CITY'].apply(str)
df['LIVE_CITY_NOT_WORK_CITY'] = df['LIVE_CITY_NOT_WORK_CITY'].apply(str)
df['EXT_SOURCE_1'] = df['EXT_SOURCE_1'].apply(str)
df['EXT_SOURCE_2'] = df['EXT_SOURCE_2'].apply(str)
df['YEARS_BUILD_MEDI'] = df['YEARS_BUILD_MEDI'].apply(str)
df['COMMONAREA_MEDI'] = df['COMMONAREA_MEDI'].apply(str)
df['ELEVATORS_MEDI'] = df['ELEVATORS_MEDI'].apply(str)
df['ENTRANCES_MEDI'] = df['ENTRANCES_MEDI'].apply(str)
df['FLOORSMAX_MEDI'] = df['FLOORSMAX_MEDI'].apply(str)
df['FLOORSMIN_MEDI'] = df['FLOORSMIN_MEDI'].apply(str)
df['LANDAREA_MEDI'] = df['LANDAREA_MEDI'].apply(str)
df['LIVINGAPARTMENTS_MEDI'] = df['LIVINGAPARTMENTS_MEDI'].apply(str)
df['LIVINGAREA_MEDI'] = df['LIVINGAREA_MEDI'].apply(str)
df['NONLIVINGAPARTMENTS_MEDI'] = df['NONLIVINGAPARTMENTS_MEDI'].apply(str)
df['NONLIVINGAREA_MEDI'] = df['NONLIVINGAREA_MEDI'].apply(str)
df['TOTALAREA_MODE'] = df['TOTALAREA_MODE'].apply(str)
df['DAYS_LAST_PHONE_CHANGE'] = df['DAYS_LAST_PHONE_CHANGE'].apply(str)
df['AMT_REQ_CREDIT_BUREAU_HOUR'] = df['AMT_REQ_CREDIT_BUREAU_HOUR'].apply(str)
df['AMT_REQ_CREDIT_BUREAU_DAY'] = df['AMT_REQ_CREDIT_BUREAU_DAY'].apply(str)
df['AMT_REQ_CREDIT_BUREAU_WEEK'] = df['AMT_REQ_CREDIT_BUREAU_WEEK'].apply(str)
df['AMT_REQ_CREDIT_BUREAU_MON'] = df['AMT_REQ_CREDIT_BUREAU_MON'].apply(str)
df['AMT_REQ_CREDIT_BUREAU_QRT'] = df['AMT_REQ_CREDIT_BUREAU_QRT'].apply(str)
df['OBS_30_CNT_SOCIAL_CIRCLE'] = df['OBS_30_CNT_SOCIAL_CIRCLE'].apply(str)
df['DEF_30_CNT_SOCIAL_CIRCLE'] = df['DEF_30_CNT_SOCIAL_CIRCLE'].apply(str)

In [52]:
df_snowflake=session.createDataFrame(
        df.values.tolist(),
        schema=df.columns.tolist())
df_snowflake.write.mode("overwrite").save_as_table("FDC_DATA_MANISH.PUBLIC.CRA_APPLICATION_OUTPUT_DECISION_V1")

In [53]:
df =  pd.read_csv('/data/Lumin_Dump_CreditRiskAnalytics_v2.csv')

In [54]:
df['APPL_MONTH'] = pd.to_datetime(df['APPL_MONTH'], format='ISO8601')

In [55]:
df['HOUSETYPE_MODE'].fillna('Missing', inplace=True)
df['WALLSMATERIAL_MODE'].fillna('Missing', inplace=True)
df['EMERGENCYSTATE_MODE'].fillna('Missing', inplace=True)
df['OCCUPATION_TYPE'].fillna('Missing', inplace=True)
df['FONDKAPREMONT_MODE'].fillna('Missing', inplace=True)
df['NAME_TYPE_SUITE'].fillna('Missing', inplace=True)

In [56]:
df['RECORD_COUNT'] = df['RECORD_COUNT'].apply(str)
df['APPL_MONTH'] = df['APPL_MONTH'].apply(str)
df['SK_ID_CURR'] = df['SK_ID_CURR'].apply(str)
df['TARGET'] = df['TARGET'].apply(str)
df['FLAG_MOBIL'] = df['FLAG_MOBIL'].apply(str)
df['FLAG_EMP_PHONE'] = df['FLAG_EMP_PHONE'].apply(str)
df['FLAG_WORK_PHONE'] = df['FLAG_WORK_PHONE'].apply(str)
df['FLAG_CONT_MOBILE'] = df['FLAG_CONT_MOBILE'].apply(str)
df['FLAG_PHONE'] = df['FLAG_PHONE'].apply(str)
df['FLAG_EMAIL'] = df['FLAG_EMAIL'].apply(str)
df['REGION_RATING_CLIENT_W_CITY'] = df['REGION_RATING_CLIENT_W_CITY'].apply(str)
df['HOUR_APPR_PROCESS_START'] = df['HOUR_APPR_PROCESS_START'].apply(str)
df['REG_REGION_NOT_LIVE_REGION'] = df['REG_REGION_NOT_LIVE_REGION'].apply(str)
df['REG_REGION_NOT_WORK_REGION'] = df['REG_REGION_NOT_WORK_REGION'].apply(str)
df['LIVE_REGION_NOT_WORK_REGION'] = df['LIVE_REGION_NOT_WORK_REGION'].apply(str)
df['REG_CITY_NOT_LIVE_CITY'] = df['REG_CITY_NOT_LIVE_CITY'].apply(str)
df['REG_CITY_NOT_WORK_CITY'] = df['REG_CITY_NOT_WORK_CITY'].apply(str)
df['LIVE_CITY_NOT_WORK_CITY'] = df['LIVE_CITY_NOT_WORK_CITY'].apply(str)
df['EXT_SOURCE_1'] = df['EXT_SOURCE_1'].apply(str)
df['EXT_SOURCE_2'] = df['EXT_SOURCE_2'].apply(str)
df['YEARS_BUILD_MEDI'] = df['YEARS_BUILD_MEDI'].apply(str)
df['COMMONAREA_MEDI'] = df['COMMONAREA_MEDI'].apply(str)
df['ELEVATORS_MEDI'] = df['ELEVATORS_MEDI'].apply(str)
df['ENTRANCES_MEDI'] = df['ENTRANCES_MEDI'].apply(str)
df['FLOORSMAX_MEDI'] = df['FLOORSMAX_MEDI'].apply(str)
df['FLOORSMIN_MEDI'] = df['FLOORSMIN_MEDI'].apply(str)
df['LANDAREA_MEDI'] = df['LANDAREA_MEDI'].apply(str)
df['LIVINGAPARTMENTS_MEDI'] = df['LIVINGAPARTMENTS_MEDI'].apply(str)
df['LIVINGAREA_MEDI'] = df['LIVINGAREA_MEDI'].apply(str)
df['NONLIVINGAPARTMENTS_MEDI'] = df['NONLIVINGAPARTMENTS_MEDI'].apply(str)
df['NONLIVINGAREA_MEDI'] = df['NONLIVINGAREA_MEDI'].apply(str)
df['TOTALAREA_MODE'] = df['TOTALAREA_MODE'].apply(str)
df['DAYS_LAST_PHONE_CHANGE'] = df['DAYS_LAST_PHONE_CHANGE'].apply(str)
df['AMT_REQ_CREDIT_BUREAU_HOUR'] = df['AMT_REQ_CREDIT_BUREAU_HOUR'].apply(str)
df['AMT_REQ_CREDIT_BUREAU_DAY'] = df['AMT_REQ_CREDIT_BUREAU_DAY'].apply(str)
df['AMT_REQ_CREDIT_BUREAU_WEEK'] = df['AMT_REQ_CREDIT_BUREAU_WEEK'].apply(str)
df['AMT_REQ_CREDIT_BUREAU_MON'] = df['AMT_REQ_CREDIT_BUREAU_MON'].apply(str)
df['AMT_REQ_CREDIT_BUREAU_QRT'] = df['AMT_REQ_CREDIT_BUREAU_QRT'].apply(str)
df['OBS_30_CNT_SOCIAL_CIRCLE'] = df['OBS_30_CNT_SOCIAL_CIRCLE'].apply(str)
df['DEF_30_CNT_SOCIAL_CIRCLE'] = df['DEF_30_CNT_SOCIAL_CIRCLE'].apply(str)

In [57]:
df_snowflake=session.createDataFrame(
        df.values.tolist(),
        schema=df.columns.tolist())
df_snowflake.write.mode("overwrite").save_as_table("FDC_DATA_MANISH.PUBLIC.CRA_APPLICATION_OUTPUT_DECISION_v2")

## Once the tables are ready, please execute below query on snowflake to create APPLICATION_DATE as DATE

ALTER TABLE FDC_DATA_MANISH.PUBLIC.CRA_APPLICATION_OUTPUT_DECISION_v1 ADD COLUMN APPLICATION_DATE DATE
UPDATE FDC_DATA_MANISH.PUBLIC.CRA_APPLICATION_OUTPUT_DECISION_v1 SET APPLICATION_DATE = TRY_TO_DATE(APPL_MONTH, 'dd-mm-yyyy')

ALTER TABLE FDC_DATA_MANISH.PUBLIC.CRA_APPLICATION_OUTPUT_DECISION_v2 ADD COLUMN APPLICATION_DATE DATE;
UPDATE FDC_DATA_MANISH.PUBLIC.CRA_APPLICATION_OUTPUT_DECISION_v2 SET APPLICATION_DATE = TRY_TO_DATE(APPL_MONTH, 'dd-mm-yyyy');